In [1]:
# import needed libraries

from firebase_admin import db, credentials, storage
from dotenv import load_dotenv, find_dotenv
import firebase_admin
import pandas as pd
import os

# load the .env file
_ = load_dotenv(find_dotenv())

### Initialize Firebase

In [2]:
service_account_info ={
    "type": os.getenv('FIREBASE_TYPE'),
    "project_id": os.getenv('FIREBASE_PROJECT_ID'),
    "private_key_id": os.getenv('FIREBASE_PRIVATE_KEY_ID'),
    "private_key": os.getenv('FIREBASE_PRIVATE_KEY'),
    "client_email": os.getenv('FIREBASE_CLIENT_EMAIL'),
    "client_id": os.getenv('FIREBASE_CLIENT_ID'),
    "auth_uri": os.getenv('FIREBASE_AUTH_URI'),
    "token_uri": os.getenv('FIREBASE_TOKEN_URI'),
    "auth_provider_x509_cert_url": os.getenv('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
    "client_x509_cert_url": os.getenv('FIREBASE_CLIENT_X509_CERT_URL'),
    "universe_domain": os.getenv('FIREBASE_UNIVERSE_DOMAIN')
  }
  

* Ensure to create a Realtime database (to get URL) and storage Bucket--link to Google Billing account for access--before embarking on the code below.

In [ ]:
# connect to the firebase database and storage with my credentials

cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'coffeeshop-app-3c5cc.firebasestorage.app',
     'databaseURL': 'https://coffeeshop-app-3c5cc-default-rtdb.firebaseio.com/'
})

In [ ]:
# initialize Firebase storage BUcket

bucket = storage.bucket()

#### Upload data Processes

In [5]:
# get image path

image_folder_path = './products/images/'

In [7]:
# Get all products collection to realtime database

products_collection = db.reference('products')

In [8]:
# read the products json l-file and specify lines=True

df = pd.read_json('products/products.jsonl',lines=True)
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


#### Function to upload images to Storage

In [9]:
# test for the image_path parameter in the function below

image_path = 'products/images/Latte.jpg'
image = image_path.split('/')
image1 = image[-1]

print(image, image1)

['products', 'images', 'Latte.jpg'] Latte.jpg


In [10]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f'product_images/{image_name}')
    # Upload image
    blob.upload_from_filename(image_path)
    # Make the image publicly accessible and get its URL
    blob.make_public()
    return blob.public_url

In [11]:
# just for test to understand the codes after this one

for index, row in df.iterrows():
    print (index, row)
    #print(index, row['name'])

0 name                                                  Cappuccino
category                                                  Coffee
description    A rich and creamy cappuccino made with freshly...
ingredients                  [Espresso, Steamed Milk, Milk Foam]
price                                                        4.5
rating                                                       4.7
image_path                                        cappuccino.jpg
Name: 0, dtype: object
1 name                                          Jumbo Savory Scone
category                                                  Bakery
description    Deliciously flaky and buttery, this jumbo savo...
ingredients    [Flour, Butter, Cheese, Herbs, Baking Powder, ...
price                                                       3.25
rating                                                       4.3
image_path                                      SavoryScone.webp
Name: 1, dtype: object
2 name                                  

In [12]:
# loop over the dataframe products.jsonl

for index, row in df.iterrows():
    print(index, row['name'])
    
    image_path = os.path.join(image_folder_path,row['image_path'])
    
    image_url = upload_image(bucket,image_path)
    product_data = row.to_dict()
    product_data.pop('image_path')
    product_data['image_url'] = image_url
    
    # Add to Firestore
    products_collection.push().set(product_data)

0 Cappuccino
1 Jumbo Savory Scone
2 Latte
3 Chocolate Chip Biscotti
4 Espresso shot
5 Hazelnut Biscotti
6 Chocolate Croissant
7 Dark chocolate
8 Cranberry Scone
9 Croissant
10 Almond Croissant
11 Ginger Biscotti
12 Oatmeal Scone
13 Ginger Scone
14 Chocolate syrup
15 Hazelnut syrup
16 Carmel syrup
17 Sugar Free Vanilla syrup


* Note: after running the above, go to Firebase Storage and Realtime database to view the uploaded data. You can also get the link URL